**Format PD GWAS summary statistics for FUMA**

## Set up and install packages if they are missing

In [15]:
# Utility routine for installing packages
install_if_missing <- function(packages) {
    if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
        install.packages(setdiff(packages, rownames(installed.packages())))
    }
}

In [16]:
install_if_missing(c('tidyverse', 'viridis', 'ggthemes', 'pryr', 'skimr', 'testthat', 'reticulate'))

Installing packages into ‘/home/jupyter-user/notebooks/packages’
(as ‘lib’ is unspecified)

also installing the dependency ‘viridisLite’




## Load libraries

In [17]:
library(viridis)    # A nice color scheme for plots.
library(ggthemes)   # Common themes to change the look and feel of plots.
library(scales)     # Graphical scales map data to aesthetics in plots.
library(testthat)   # Testing functions.
library(assertthat) # Assertion functions.
library(pryr)       # Memory usage functions.
library(skimr)      # Summary statistics for dataframes.
library(bigrquery)  # BigQuery R client.
library(tidyverse)  # Data wrangling packages.
library(reticulate)  # R Interface to Python

library(Ronaldo)    # Leonardo R package.

library(data.table) # Data Table package for faster reading and processing

# Load reticulate for calling the FireCloud Python API
library(reticulate)

# Load ggplot2 for graphs
library(ggplot2)

# Load biqrquery for interacting with BigQuery
library(bigrquery)

Loading required package: viridisLite


Attaching package: ‘scales’


The following object is masked from ‘package:viridis’:

    viridis_pal


The following object is masked from ‘package:purrr’:

    discard


The following object is masked from ‘package:readr’:

    col_factor



Attaching package: ‘testthat’


The following object is masked from ‘package:dplyr’:

    matches


The following object is masked from ‘package:purrr’:

    is_null


The following object is masked from ‘package:tidyr’:

    matches



Attaching package: ‘assertthat’


The following object is masked from ‘package:tibble’:

    has_name


Registered S3 method overwritten by 'pryr':
  method      from
  print.bytes Rcpp


Attaching package: ‘pryr’


The following objects are masked from ‘package:purrr’:

    compose, partial


The following object is masked from ‘package:data.table’:

    address



Attaching package: ‘skimr’


The following object is masked from ‘package:testthat’:

    matches




In [27]:
#Install miniconda otherwise the import firecloud.api does not work
reticulate::install_miniconda()

* Downloading 'https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh' ...

* Installing Miniconda -- please wait a moment ...

* Miniconda has been successfully installed at '/home/jupyter-user/.local/share/r-miniconda'.



[1] "/home/jupyter-user/.local/share/r-miniconda"

## Set up environment variables

In [44]:
BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')
WORKSPACE_NAMESPACE <- Sys.getenv('WORKSPACE_NAMESPACE')
WORKSPACE_NAME <- Sys.getenv('WORKSPACE_NAME')
WORKSPACE_BUCKET <-  Sys.getenv('WORKSPACE_BUCKET')

In [45]:
#This is not working for me but not needed as we are not accessing the AMP clinical data
fapi <- import("firecloud.api")
WORKSPACE_ATTRIBUTES <- fapi$get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME)$json()$workspace$attributes

GS_CLINICAL_RELEASE_PATH <- WORKSPACE_ATTRIBUTES['gs_path_amp_pd_clinical_release']
str_glue('GS_CLINICAL_RELEASE_PATH = {GS_CLINICAL_RELEASE_PATH}')

ERROR: Error in py_module_import(module, convert = convert): ModuleNotFoundError: No module named 'firecloud'

Detailed traceback: 
  File "/usr/local/lib/R/site-library/reticulate/python/rpytools/loader.py", line 24, in _import_hook
    level=level



## Set up utility functions

In [20]:
# Utility routine for printing a shell command before executing it
shell_do <- function(command) {
    print(paste('Executing: ', command))
    system(command, intern = TRUE)
}

In [21]:
# Utility routines for reading files from Google Cloud Storage
gcs_read_file <- function(path) {
    pipe(str_glue('gsutil -u {BILLING_PROJECT_ID} cat {path}'))
}
gcs_read_csv <- function(path, sep=',') {
    readr::read_csv(gcs_read_file(path))
}

# Utility routines for reading files from Google BigQuery
bq_query <- function(query) {
    # Return the contents of a query against BigQuery    
    return(bigrquery::bq_table_download(
        bigrquery::bq_project_query(BILLING_PROJECT_ID, query = query)))
}

## Read in PD GWAS summary statistics

In [2]:
data <- fread("~/bin/data_temp/nallsEtAl2019_excluding23andMe_allVariants.tab", header = T)

In [19]:
head(data)

SNP,A1,A2,freq,b,se,p,N_cases,N_controls,chr,bp,chr_new,total_N
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<int>
chr11:88249377,T,C,0.9931,0.1575,0.1783,0.3771,7161,5356,chr11,88249377,11,12517
chr1:60320992,A,G,0.9336,0.0605,0.0456,0.1846,26421,442271,chr1,60320992,1,468692
chr2:18069070,T,C,0.9988,-0.6774,1.3519,0.6163,582,905,chr2,18069070,2,1487
chr8:135908647,A,G,0.2081,-0.0358,0.0273,0.1887,26421,442271,chr8,135908647,8,468692
chr12:3871714,A,C,0.9972,0.1489,1.0636,0.8886,749,658,chr12,3871714,12,1407
chr16:77148858,A,G,0.9976,-0.1213,0.3874,0.7543,6248,4391,chr16,77148858,16,10639


## Split SNP column into chr and bp

In [4]:
#Data table
setDT(data)[, c("chr", "bp") := tstrsplit(SNP, ":")]

In [ ]:
#Dplyr solution taking too long
#data <- data %>%
#    separate(SNP, into = c("chr", "bp"), sep = ":")

In [5]:
head(data)

SNP,A1,A2,freq,b,se,p,N_cases,N_controls,chr,bp
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>
chr11:88249377,T,C,0.9931,0.1575,0.1783,0.3771,7161,5356,chr11,88249377
chr1:60320992,A,G,0.9336,0.0605,0.0456,0.1846,26421,442271,chr1,60320992
chr2:18069070,T,C,0.9988,-0.6774,1.3519,0.6163,582,905,chr2,18069070
chr8:135908647,A,G,0.2081,-0.0358,0.0273,0.1887,26421,442271,chr8,135908647
chr12:3871714,A,C,0.9972,0.1489,1.0636,0.8886,749,658,chr12,3871714
chr16:77148858,A,G,0.9976,-0.1213,0.3874,0.7543,6248,4391,chr16,77148858


## Remove 'chr' text from chromosome column

In [6]:
data <- data %>%
    mutate(chr_new = gsub("chr", "", chr))

In [7]:
head(data)

SNP,A1,A2,freq,b,se,p,N_cases,N_controls,chr,bp,chr_new
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>
chr11:88249377,T,C,0.9931,0.1575,0.1783,0.3771,7161,5356,chr11,88249377,11
chr1:60320992,A,G,0.9336,0.0605,0.0456,0.1846,26421,442271,chr1,60320992,1
chr2:18069070,T,C,0.9988,-0.6774,1.3519,0.6163,582,905,chr2,18069070,2
chr8:135908647,A,G,0.2081,-0.0358,0.0273,0.1887,26421,442271,chr8,135908647,8
chr12:3871714,A,C,0.9972,0.1489,1.0636,0.8886,749,658,chr12,3871714,12
chr16:77148858,A,G,0.9976,-0.1213,0.3874,0.7543,6248,4391,chr16,77148858,16


## Make a column for total N (cases + controls)

In [8]:
data <- data %>%
    mutate(total_N = N_cases + N_controls)

In [9]:
head(data)

SNP,A1,A2,freq,b,se,p,N_cases,N_controls,chr,bp,chr_new,total_N
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<int>
chr11:88249377,T,C,0.9931,0.1575,0.1783,0.3771,7161,5356,chr11,88249377,11,12517
chr1:60320992,A,G,0.9336,0.0605,0.0456,0.1846,26421,442271,chr1,60320992,1,468692
chr2:18069070,T,C,0.9988,-0.6774,1.3519,0.6163,582,905,chr2,18069070,2,1487
chr8:135908647,A,G,0.2081,-0.0358,0.0273,0.1887,26421,442271,chr8,135908647,8,468692
chr12:3871714,A,C,0.9972,0.1489,1.0636,0.8886,749,658,chr12,3871714,12,1407
chr16:77148858,A,G,0.9976,-0.1213,0.3874,0.7543,6248,4391,chr16,77148858,16,10639


## Select relevant columns and export as .txt for FUMA

In [10]:
export <- data %>%
    select(chr_new, bp, p, A1, A2, b, se, total_N) %>%
    rename(chr = chr_new)

In [11]:
head(export)

chr,bp,p,A1,A2,b,se,total_N
<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<int>
11,88249377,0.3771,T,C,0.1575,0.1783,12517
1,60320992,0.1846,A,G,0.0605,0.0456,468692
2,18069070,0.6163,T,C,-0.6774,1.3519,1487
8,135908647,0.1887,A,G,-0.0358,0.0273,468692
12,3871714,0.8886,A,C,0.1489,1.0636,1407
16,77148858,0.7543,A,G,-0.1213,0.3874,10639


In [35]:
fwrite(export, "~/bin/data_temp/PDGWAS_for_FUMA_MT.txt", quote = F, col.names = T, row.names = F, sep = "\t")

## Zip txt file and export formatted file to workspace bucket

In [54]:
system('ls ~/bin/data_temp/', intern=TRUE) #List files in ~/bin/data_temp/

[1] "nallsEtAl2019_excluding23andMe_allVariants.tab"    
[2] "nallsEtAl2019_excluding23andMe_allVariants.tab.zip"
[3] "PDGWAS_for_FUMA_MT.txt"                            
[4] "PDGWAS_for_FUMA_MT.txt.zip"                        
[5] "test.zip"

In [53]:
system('zip ~/bin/data_temp/PDGWAS_for_FUMA_MT.txt.zip ~/bin/data_temp/PDGWAS_for_FUMA_MT.txt', intern=TRUE)
#Zip txt file 

[1] "  adding: home/jupyter-user/bin/data_temp/PDGWAS_for_FUMA_MT.txt (deflated 61%)"

In [46]:
shell_do(str_glue('gsutil -u {BILLING_PROJECT_ID} cp -r ~/bin/data_temp/PDGWAS_for_FUMA_MT.txt.zip {WORKSPACE_BUCKET}'))
#Copy the final formatted txt.zip file to the workspace bucket so we can download it

[1] "Executing:  gsutil -u gp2-ipdgc-hackathon cp -r ~/bin/data_temp/PDGWAS_for_FUMA_MT.txt.zip gs://fc-secure-b9f9d17f-b38c-407d-85e0-7a759f13cea0"


character(0)